<a href="https://colab.research.google.com/github/Akshith213/Technocolabs/blob/main/Toxic_Text_Classification_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mini Project (Toxic comment classification)
* By Kota Akshith Reddy
* Instructions on how to run this file is available in Readme section of my Github repository.

In [25]:
#Installing PyDrive 
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [29]:
#Getting the links for train and test datasets
link_train = 'https://drive.google.com/file/d/1eD16ULKuFiWP57TwIydXVJJkS3718YBD/view?usp=sharing'
link_test='https://drive.google.com/file/d/1OI8WQBdfRAhRsRcoHr3r_O3Sl9zOxT3w/view?usp=sharing'
link_model='https://drive.google.com/file/d/1VdPVm9u--knE7cOHxh7kHaG_2k_dkXlI/view?usp=sharing'
id_train = '1eD16ULKuFiWP57TwIydXVJJkS3718YBD'
id_test = '1OI8WQBdfRAhRsRcoHr3r_O3Sl9zOxT3w'
id_model = '1VdPVm9u--knE7cOHxh7kHaG_2k_dkXlI'


In [30]:
#Importing train and test datasets
downloaded = drive.CreateFile({'id':id_train}) 
downloaded.GetContentFile('train.csv')  
df= pd.read_csv('train.csv')
downloaded = drive.CreateFile({'id':id_test}) 
downloaded.GetContentFile('test.csv')  
test= pd.read_csv('test.csv')


In [34]:
downloaded = drive.CreateFile({'id':id_model}) 
downloaded.GetContentFile('model_roberta.sav')  
model=pickle.load(open('model_roberta.sav', 'rb'))

In [35]:
model

In [1]:
#Importing necessary packages
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import nltk
import numpy as np
import random
import json
import pickle
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
import numpy as np
from numpy import dot
from numpy.linalg import norm
import string
import warnings
!pip install simpletransformers
from simpletransformers.classification import MultiLabelClassificationModel
! pip install flask-ngrok


     |████████████████████████████████| 204kB 5.6MB/s 
     |████████████████████████████████| 2.9MB 14.4MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 1.4MB 36.5MB/s 
     |████████████████████████████████| 7.4MB 44.6MB/s 
     |████████████████████████████████| 317kB 45.0MB/s 
     |████████████████████████████████| 1.1MB 38.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 1.8MB 41.9MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 112kB 41.6MB/s 
     |████████████████████████████████| 163kB 46.2MB/s 
     |████████████████████████████████| 4.5MB 42.9MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 133kB 43.9MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 122kB 44.3MB/s 

In [2]:
import nltk
nltk.download('stopwords')
warnings.filterwarnings("ignore")
stop_words=stopwords.words('english')
nltk.download('wordnet')
!pip install contractions
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 317kB 4.4MB/s 
     |████████████████████████████████| 245kB 7.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81695 sha256=a09dddbbc4d0d15531b5a940341a5a166b6de50fa096a42f0fcb5596e598592e
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [3]:
#Function to preprocess data
def text_preprocessing(df):
  #df['patterns']=df['patterns'].apply(', '.join)
  df['comment_text'] = df['comment_text'].apply(lambda i:' '.join(i.lower() for i in i.split()))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i in stop_words))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i in string.punctuation))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i.isdigit()))
  df['comment_text']= df['comment_text'].str.replace('[^\w\s]','')
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(WordNetLemmatizer().lemmatize(w) for w in i.split()))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(contractions.fix(w) for w in i.split()))
  #return df['comment_text']


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))


In [8]:
text_preprocessing(df)

In [9]:
df['text']=df['comment_text']

In [10]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww match background colour I am seemingly st...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",daww match background colour I am seemingly st...
2,000113f07ec002fd,hey man I am really trying edit war guy consta...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey man I am really trying edit war guy consta...
3,0001b41b1c6bb37e,can not make real suggestion improvement wonde...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",can not make real suggestion improvement wonde...
4,0001d958c54c6e35,you sir hero chance remember page that is on,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",you sir hero chance remember page that is on


In [11]:
train_df,eval_df=train_test_split(df,test_size=0.2)

In [12]:
#model=MultiLabelClassificationModel('roberta','roberta-base',num_labels=6,args={'train_batch_size':256,'gradient_accumulation_steps':16,'learning_rate':3e-5,'num_train_epochs':3,'max_seq_length':64})

In [13]:
train_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
2469,06960372c6ea8fa6,actually fully arabic islamic correct spelling...,1,0,0,0,1,1,"(1, 0, 0, 0, 1, 1)",actually fully arabic islamic correct spelling...
88575,ecf3f3110b15f540,motherfucker look you done reference code mess...,1,0,1,0,0,0,"(1, 0, 1, 0, 0, 0)",motherfucker look you done reference code mess...
153403,9c79a0971c445075,welcome here I am welcome everywhere,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",welcome here I am welcome everywhere
107285,3d81e3f94eb569b0,oppose title clumsy need changing article need...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",oppose title clumsy need changing article need...
5117,0d8fc205e235e213,hello making potentially controversial edits r...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hello making potentially controversial edits r...


In [14]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww match background colour I am seemingly st...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",daww match background colour I am seemingly st...
2,000113f07ec002fd,hey man I am really trying edit war guy consta...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey man I am really trying edit war guy consta...
3,0001b41b1c6bb37e,can not make real suggestion improvement wonde...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",can not make real suggestion improvement wonde...
4,0001d958c54c6e35,you sir hero chance remember page that is on,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",you sir hero chance remember page that is on


In [15]:
#model.train_model(train_df)

In [16]:
eval_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
1855,0503eb6ae953ae61,hey tomorrow tony happy let celebrate get pc c...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey tomorrow tony happy let celebrate get pc c...
39802,6a48d8cd59602a9d,number smaller course matter factual innaccura...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",number smaller course matter factual innaccura...
156770,d3526bba565b944e,think need revised way written suggests anothe...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",think need revised way written suggests anothe...
102363,23c9e76fb18491d8,ok let move somewhere else first answer questi...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",ok let move somewhere else first answer questi...
75352,c9935945d50f5b72,language referred article central lake plain c...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",language referred article central lake plain c...


In [17]:
#filename = '/content/drive/MyDrive/Technocolabs/Mini project/model_roberta.sav'
#model = pickle.load(open(filename, 'rb'))

In [18]:
#preds,outputs=model.predict(eval_df['text'].tolist())

In [19]:
#pickle.dump(model, open(filename, 'wb'))

In [20]:
def get_input():
  inp=input()
  df_inp=pd.DataFrame()
  df_inp['comment_text']=[inp]
  #text_preprocessing(df_inp)
  df_inp['text']=df_inp['comment_text']
  df_inp.drop('comment_text',axis=1,inplace=True)
  return df_inp



In [36]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import pandas as pd
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)

def get_input(inp):
    df= pd.DataFrame()
    df['comment_text'] = inp
    text_preprocessing(df)
    df['text'] = df['comment_text']
    df.drop('comment_text', axis=1, inplace=True)
    return df

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    inp_text = [str(x) for x in request.form.values()]
    df=get_input(inp_text)
    preds, outputs = model.predict(df['text'].tolist())
    d={}
    d['toxic']=outputs[0][0]
    d['severe toxic']=outputs[0][1]
    d['obscene']=outputs[0][2]
    d['threat']=outputs[0][3]
    d['insult']=outputs[0][4]
    d['identity hate']=outputs[0][5]
    dc={}
    dc['toxic']=preds[0][0]
    dc['severe toxic']=preds[0][1]
    dc['obscene']=preds[0][2]
    dc['threat']=preds[0][3]
    dc['insult']=preds[0][4]
    dc['identity hate']=preds[0][5]


    return render_template('index.html', prediction_probabilities='Prediction probabilities are {}'.format(d),predicted_classes='Predicted classes are {}'.format(dc))


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cbaa7de7d6ef.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Dec/2020 12:00:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2020 12:00:04] "GET /favicon.ico HTTP/1.1" 404 -


127.0.0.1 - - [12/Dec/2020 12:00:07] "POST /predict HTTP/1.1" 200 -
